# Imports

In [1]:
import gc
import torch
import requests
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

# Llama 3.2

In [2]:
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
image = Image.open("/home/ilyass/reference3.png")

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "Give me the list of the objects present in the image their respective upper-left and bottom-right corners of the bounding box relative to the image size using this format [(object_name, upper-left corner, bottom-right corner)]"}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(image, input_text, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_new_tokens=1e6)
print(processor.decode(output[0][inputs["input_ids"].shape[-1]:]))

**Objects in the Image:**

*   Blue Table: [(0.07, 0.06), (0.93, 0.44)]
*   Green Cube: [(0.32, 0.57), (0.73, 0.79)]
*   Brown Floor Tiles: [(0.00, 0.00), (1.00, 0.98)]<|eot_id|>


In [11]:
image = Image.open("/home/ilyass/reference3.png")

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "Given the image, what are the sequence of actions a person should perform to place the green cube on top of the table? Your answers should have a format similar to <verb> + <determinant> + <object>, for example 'pick the mugh' ."}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(image, input_text, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_new_tokens=1e6)
print(processor.decode(output[0][inputs["input_ids"].shape[-1]:]))

To place the green cube on top of the table, a person should perform the following actions:

1. Pick the green cube.
2. Move the green cube to the table.
3. Place the green cube on top of the table.<|eot_id|>


In [5]:
# print(processor.decode(output[0]))

In [9]:
def clean_gpu():
    with torch.no_grad():
        torch.cuda.empty_cache()

In [10]:
clean_gpu()
del input_text, inputs, output

gc.collect()
torch.cuda.empty_cache()

In [4]:
image = Image.open("/home/ilyass/Pictures/Screenshots/reference.png")

In [9]:
image.width * 0.65

670.15

In [10]:
image.height * 0.65

507.0